In [13]:
!pip install mlforecast
!pip install numba
!pip install window_ops

ERROR: Could not find a version that satisfies the requirement rolling_mean (from versions: none)
ERROR: No matching distribution found for rolling_mean


In [15]:
import yfinance as yf
import pandas as pd

# Coleta de dados históricos
data = yf.download('AAPL', start='2010-01-01', end='2023-01-01')
data.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,7.622500,7.660714,7.585000,7.643214,6.470740,493729600
2010-01-05,7.664286,7.699643,7.616071,7.656429,6.481928,601904800
2010-01-06,7.656429,7.686786,7.526786,7.534643,6.378825,552160000
2010-01-07,7.562500,7.571429,7.466071,7.520714,6.367033,477131200
2010-01-08,7.510714,7.571429,7.466429,7.570714,6.409363,447610800


In [16]:
# Selecionar e renomear colunas
data2 = data.reset_index()[['Date', 'Close']]#reseta os indices para transformar-los em colunas normais
data2 = data2.rename(columns={'Date': 'ds', 'Close': 'y'})

# Adicionar coluna 'unique_id' com valor constante 'AAPL'
data2['unique_id'] = 'AAPL'

#lembrando que essa parte é bastante importante para manter o padrão das bibliotecas do grupo Nixtla, que é o criador da mlforecast, vamos renomear as colunas para ds (data), y (alvo) e unique_id (family).

# Mostrar as primeiras linhas dos dados
print(data2.head())


          ds         y unique_id
0 2010-01-04  7.643214      AAPL
1 2010-01-05  7.656429      AAPL
2 2010-01-06  7.534643      AAPL
3 2010-01-07  7.520714      AAPL
4 2010-01-08  7.570714      AAPL


In [17]:
train = data2.loc[data2['ds'] < '2017-01-01']
valid = data2.loc[(data2['ds'] >= '2017-01-01') & (data2['ds'] < '2017-04-01')]


In [18]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

models = [RandomForestRegressor(random_state=0, n_estimators=100),
          ExtraTreesRegressor(random_state=0, n_estimators=100)]

CONCEITOS IMPORTANTES

1-LAGS:
Lags são simplesmente valores passados da série temporal que você inclui como features para o modelo.

Por exemplo, se você está tentando prever a demanda de um produto daqui a uma semana, pode incluir a demanda do mesmo dia da semana, mas na semana anterior como feature.

Na prática a demanda de um produto em um dia da semana acaba sendo similar em semanas diferentes, então essa feature pode ser útil para o modelo.

Você também pode criar lags mais longos, como o valor da série temporal no mesmo dia do ano anterior (imagine que você está tentando prever a demanda de um produto no Natal ou Black Friday).

É importante testar vários valores de lags para encontrar o que funciona melhor para o seu problema específico.

Ninguém pode te dizer quais são os melhores lags para o seu problema sem testar.



2-AGREGAÇÕES TEMPORAIS:
As agregações temporais são operações aplicadas a uma janela de observações da série temporal.

Por exemplo, calcular a média de vendas diárias dos últimos 7 dias, ou o desvio padrão dos retornos de um ativo financeiro nas últimas 4 semanas.

Essas agregações podem ajudar o modelo a identificar tendências e padrões na série temporal.

Assim como no caso dos lags, é importante experimentar várias agregações temporais e vários tamanhos de janelas diferentes, para encontrar o que funciona melhor para o seu problema específico.



3-COMPONENTES DE DATA:
Os componentes da data são valores que você extrai de uma data ou hora específica.

Por exemplo, você pode incluir a hora do dia, o dia da semana, o mês, a estação do ano, etc.

Essas features podem ajudar o modelo a identificar padrões sazonais e cíclicos na série temporal.

Preciso dizer que é importante testar vários componentes de data diferentes para encontrar o que funciona melhor para o seu problema específico?

4-DIFERENÇAS:
As diferenças são o resultado da subtração entre valores da série temporal.

Por exemplo, você pode incluir a diferença entre a demanda de hoje e a demanda de ontem.

Mesmo com os lags, as diferenças podem trazer informações mais detalhadas para o modeo e ajudá-lo a identificar os padrões com mais facilidade.

A mesma recomendação de testar “diferenças diferentes” 🙂 vale aqui.

In [19]:
#função para calcular diferenças entre series temporais em um determinado lag
from numba import njit
import numpy as np

@njit
def diff(x, lag):
    x2 = np.full_like(x, np.nan)
    for i in range(lag, len(x)):
        x2[i] = x[i] - x[i-lag]
    return x2

In [24]:
from mlforecast import MLForecast
#from window_ops import rolling_mean

import pandas as pd
#usaremos essa função,pois por algum motivo não estou conseguindo importar o rolling_mean do window_ops
def rolling_mean(x, window):
    return pd.Series(x).rolling(window=window).mean().values


model = MLForecast(
    models=models,
    freq='D',
    lags=[1, 7, 14],  # Lags de 1, 7 e 14 dias
    lag_transforms={
        1: [(rolling_mean, 3), (rolling_mean, 7), (rolling_mean, 28), (diff, 1), (diff, 7)]
    },
    date_features=['dayofweek'],
    num_threads=6
)

In [32]:
model.fit(train, id_col='unique_id', time_col='ds', target_col='y', static_features=[])
p = model.predict(horizon=90, dynamic_dfs=[valid[['unique_id', 'ds', 'Adj Close	']]])
p = p.merge(valid[['unique_id', 'ds', 'y']], on=['unique_id', 'ds'], how='left')

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
non-precise type pyobject
During: typing of argument at /usr/local/lib/python3.10/dist-packages/mlforecast/grouped_array.py (18)

File "../usr/local/lib/python3.10/dist-packages/mlforecast/grouped_array.py", line 18:
# %% ../nbs/grouped_array.ipynb 2
@njit(nogil=True)
^ 

This error may have been caused by the following argument(s):
- argument 4: Cannot determine Numba type of <class 'function'> 

This error may have been caused by the following argument(s):
- argument 4: Cannot determine Numba type of <class 'function'> 

This error may have been caused by the following argument(s):
- argument 4: Cannot determine Numba type of <class 'function'> 

This error may have been caused by the following argument(s):
- argument 4: Cannot determine Numba type of <class 'function'> 

This error may have been caused by the following argument(s):
- argument 4: Cannot determine Numba type of <class 'function'> 

This error may 